# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import re
from nltk import download
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.stem import  WordNetLemmatizer

In [ ]:
download('stopwords')
download('wordnet')
download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Amazon Reviews/AMAZON_FASHION_v4.parquet')

In [ ]:
new = pd.concat([df[df.sentiment == 0].head(10000), df[df.sentiment == 1].head(10000)], ignore_index=True)
del df

In [ ]:
new.reviewLength.quantile(0.8)

50.0

# Modeling

In [4]:
metrics = pd.DataFrame()

In [12]:
df = pd.read_parquet('/content/drive/MyDrive/Amazon Reviews/AMAZON_FASHION_v5.parquet')
df.shape

(20000, 51)

In [13]:
df.columns

Index(['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10',
       'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19',
       'pc20', 'pc21', 'pc22', 'pc23', 'pc24', 'pc25', 'pc26', 'pc27', 'pc28',
       'pc29', 'pc30', 'pc31', 'pc32', 'pc33', 'pc34', 'pc35', 'pc36', 'pc37',
       'pc38', 'pc39', 'pc40', 'pc41', 'pc42', 'pc43', 'pc44', 'pc45', 'pc46',
       'pc47', 'pc48', 'pc49', 'pc50', 'sentiment'],
      dtype='object')

In [14]:
X = df.drop(columns=['sentiment'])
y = df.sentiment
del df

In [33]:
def evaluate(yhat, y, model):
    global metrics
    metrics = pd.concat([metrics, pd.DataFrame(
        {
           'model': [model, ],
           'accuracy_score': [accuracy_score(y_pred=yhat, y_true=y),],
           'f1_score': [f1_score(y_pred=yhat, y_true=y),],
           'precision_score': [precision_score(y_pred=yhat, y_true=y),],
           'recall_score': [recall_score(y_pred=yhat, y_true=y),],
        }
    )]
              )

## Naive Bayes

In [40]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay

In [35]:
gnb = GaussianNB()
gnb.fit(X, y)

GaussianNB()

In [38]:
yhat = gnb.predict(X)
evaluate(yhat, y, model='GNB')

In [39]:
metrics

,model,accuracy_score,f1_score,precision_score,recall_score
0,GNB,0.7526,0.772652,0.714723,0.8408
0,GNB,0.7526,0.772652,0.714723,0.8408


## RandomForest

In [41]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [44]:
rf = RandomForestClassifier()
grid = RandomizedSearchCV(
    rf,
    param_distributions={
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy'],
        'max_depth': [2,3,5],
        'max_features': ['sqrt', 'log'],
        'max_leaf_nodes': [10, 15],
        'n_estimators': [100, 50, 150],
        'random_state': [42,],
        },
    cv=5,
    scoring='accuracy'
)

In [45]:
grid.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 3, 5],
                                        'max_features': ['sqrt', 'log'],
                                        'max_leaf_nodes': [10, 15],
                                        'n_estimators': [100, 50, 150],
                                        'random_state': [42]},
                   scoring='accuracy')

In [47]:
rf_best = grid.best_estimator_

In [48]:
yhat = rf_best.predict(X)
evaluate(yhat, y, model='RF')

In [49]:
metrics

,model,accuracy_score,f1_score,precision_score,recall_score
0,GNB,0.75260,0.772652,0.714723,0.8408
0,GNB,0.75260,0.772652,0.714723,0.8408
0,RF,0.77525,0.781489,0.760382,0.8038


In [51]:
metrics.to_csv('/content/drive/MyDrive/Amazon Reviews/metrics.csv')